In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

from __future__ import division
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
matplotlib.rcParams['figure.dpi'] = 2.5 * matplotlib.rcParams['figure.dpi']

import numpy as np
import scipy.constants as sc
import math, sys, os, glob, h5py

import libstempo as T2

try:
    from IPython.display import clear_output
    have_ipython = True
except ImportError:
    have_ipython = False

In [ ]:
def figsize(scale):
    fig_width_pt = 513.17 #469.755                  # Get this from LaTeX using \the\textwidth
    inches_per_pt = 1.0/72.27                       # Convert pt to inch
    golden_mean = (np.sqrt(5.0)-1.0)/2.0            # Aesthetic ratio (you could change this)
    fig_width = fig_width_pt*inches_per_pt*scale    # width in inches
    fig_height = fig_width*golden_mean              # height in inches
    fig_size = [fig_width,fig_height]
    return fig_size

#plt.rcParams.update(plt.rcParamsDefault)
params = {'backend': 'pdf',
        'axes.labelsize': 10,
        'lines.markersize': 4,
        'font.size': 10,
        'xtick.major.size':6,
        'xtick.minor.size':3,  
        'ytick.major.size':6,
        'ytick.minor.size':3, 
        'xtick.major.width':0.5,
        'ytick.major.width':0.5,
        'xtick.minor.width':0.5,
        'ytick.minor.width':0.5,
        'lines.markeredgewidth':1,
        'axes.linewidth':1.2,
        'legend.fontsize': 7,
        'xtick.labelsize': 10,
        'ytick.labelsize': 10,
        'savefig.dpi':200,
        'path.simplify':True,
        'font.family': 'serif',
        'font.serif':'Times',
        'text.latex.preamble': [r'\usepackage{amsmath}',r'\usepackage{amsbsy}',
                                r'\DeclareMathAlphabet{\mathcal}{OMS}{cmsy}{m}{n}'],
        'text.usetex':True,
        'figure.figsize': figsize(0.5)}
plt.rcParams.update(params)

# Reading in BayesEphem analysis chains

In [ ]:
# Chains are already burnt-in, but you may want to burn-in another 5e4 samples
chains = np.load('./data/bayes_ephem_posteriors.npz')

In [ ]:
chains = dict(chains)

In [ ]:
chains.keys()

In [ ]:
fig,ax = plt.subplots()

colors= ['C0', 'C1', 'C2', 'C3']
labels = [ii.split('chain_')[1:][0] for ii in chains.keys()]

for ii,key in enumerate(chains):
    n, _, _ = plt.hist(chains[key][:,68], bins=40, histtype='step', 
               normed=True, label=labels[ii].replace('_', ' '), color=colors[ii])
    print chains[key].shape
        
plt.legend(loc='upper left',frameon=False,ncol=2)
plt.xlabel(r'$\log_{10}A_\mathrm{GWB}$')
plt.ylabel(r'PDF')
plt.yscale('log')
plt.ylim(1e-3,5.0)
plt.xlim(-18.0,-14.0)
plt.minorticks_on()
plt.tick_params(which='both',direction='in')
plt.show()

## Analyze BayesEphem model parameters

In [ ]:
# Parameter indices 69 to 79 are BayesEphem parameters
# 69 = Frame drift-rate
# 70 = Jupiter mass perturbation
# 71 = Saturn mass perturbation
# 72 = Uranus mass perturbation
# 73 = Neptune mass perturbation
# 74 = Jupiter orbit (PCA basis weight 1)
# 75 = Jupiter orbit (PCA basis weight 2)
# 76 = Jupiter orbit (PCA basis weight 3)
# 77 = Jupiter orbit (PCA basis weight 4)
# 78 = Jupiter orbit (PCA basis weight 5)
# 79 = Jupiter orbit (PCA basis weight 6)

In [ ]:
# Conversion between PCA basis and physical-parameter basis
M = np.array([[  6.53860873e+04,   9.49761902e+04,   1.21450042e+03,
         -1.77200265e+03,  -6.22949977e+02,  -7.47925821e+02],
       [ -7.75621399e+02,  -1.17021057e+03,   5.90675833e+01,
         -7.98177856e+04,   1.26269526e+02,  -2.71788472e+04],
       [  1.11908717e+03,  -5.65791141e+02,   3.35238820e+02,
         -2.17333137e+04,  -1.57642596e+02,   7.72428458e+04],
       [  1.85720158e+04,   1.86472597e+04,   3.65546134e+04,
         -5.74076339e+01,   1.63357930e+05,  -1.52072786e+01],
       [ -2.21951581e+06,  -3.03054942e+06,  -2.04049224e+05,
          5.73677705e+04,  -3.08179452e+04,   2.63676877e+04],
       [  5.65210209e+01,   1.27036237e+04,  -1.72763570e+05,
         -3.59982642e+02,   3.04507609e+04,   4.62868757e+02]])

In [ ]:
Minv = scipy.linalg.inv(M)

In [ ]:
# Code snippet to convert to physical-parameter basis

#chain_de435_noprior_phys = np.dot(Minv,chain_de435_noprior[int(5e4):,74:80].T).T
#chain_de436_noprior_phys = np.dot(Minv,chain_de436_noprior[int(5e4):,74:80].T).T
#chain_de435_prior_phys = np.dot(Minv,chain_de435[int(5e4):,74:80].T).T
#chain_de436_prior_phys = np.dot(Minv,chain_de436[int(5e4):,74:80].T).T

# BayesEphem functions

In [ ]:
day = 24 * 3600
year = 365.25 * day

SOLAR2S = sc.G / sc.c**3 * 1.98855e30
KPC2S = sc.parsec / sc.c * 1e3
MPC2S = sc.parsec / sc.c * 1e6

e_ecl = 23.43704 * np.pi / 180.0
M_ecl = np.array([[1.0, 0.0, 0.0],
                  [0.0, np.cos(e_ecl), -np.sin(e_ecl)],
                  [0.0, np.sin(e_ecl), np.cos(e_ecl)]])


def ecl2eq_vec(x):
    """
    Rotate (n,3) vector time series from ecliptic to equatorial.
    """

    return np.einsum('jk,ik->ij',M_ecl,x)


def eq2ecl_vec(x):
    """
    Rotate (n,3) vector time series from equatorial to ecliptic.
    """

    return np.einsum('kj,ik->ij',M_ecl,x)


def euler_vec(z, y, x, n):
    """
    Return (n,3,3) tensor with each (3,3) block containing an
    Euler rotation with angles z, y, x. Optionally each of z, y, x
    can be a vector of length n.
    """

    L = np.zeros((n,3,3),'d')
    cosx, sinx = np.cos(x), np.sin(x)
    L[:,0,0] = 1
    L[:,1,1] = L[:,2,2] = cosx
    L[:,1,2] = -sinx; L[:,2,1] = sinx

    N = np.zeros((n,3,3),'d')
    cosy, siny = np.cos(y), np.sin(y)
    N[:,0,0] = N[:,2,2] = cosy
    N[:,1,1] = 1
    N[:,0,2] = siny; N[:,2,0] = -siny

    ret = np.einsum('ijk,ikl->ijl',L,N)

    M = np.zeros((n,3,3),'d')
    cosz, sinz = np.cos(z), np.sin(z)
    M[:,0,0] = M[:,1,1] = cosz
    M[:,0,1] = -sinz; M[:,1,0] = sinz
    M[:,2,2] = 1

    ret = np.einsum('ijk,ikl->ijl',ret,M)

    return ret


t_offset = 55197.0


def ss_framerotate(mjd, planet, x, y, z, dz,
                   offset=None, equatorial=False):
    """
    Rotate planet trajectory given as (n,3) tensor,
    by ecliptic Euler angles x, y, z, and by z rate
    dz. The rate has units of deg/year, and is referred
    to offset 2010/1/1. dates must be given in MJD.
    """

    if equatorial:
        planet = eq2ecl_vec(planet)

    E = euler_vec(z + dz * (mjd - t_offset) / 365.25, y, x,
                  planet.shape[0])

    planet = np.einsum('ijk,ik->ij',E,planet)

    if offset is not None:
        planet = np.array(offset) + planet

    if equatorial:
        planet = ecl2eq_vec(planet)

    return planet


def dmass(earth, planet, dm_over_Msun):
    return earth + dm_over_Msun * planet


def dorbit(mjd, earth, planet, x, y, z, dz, m_over_Msun):
    E = euler_vec(z + dz * (mjd - t_offset) / 365.25 ,y, x,
                  planet.shape[0])

    dplanet = np.einsum('ijk,ik->ij',E,planet) - planet

    return earth + m_over_Msun * dplanet


def ssephem_physical_model(x, mjd, earth, jupiter, saturn,
                           uranus, neptune,
                           incJuporb=False, jup_orbmodel='orbelements', jup_orbelxyz=None, jup_mjd=None,
                           incSatorb=False, sat_orbmodel='orbelements', sat_orbelxyz=None, sat_mjd=None,
                           equatorial=True):
    # model with argument x, see below for priors.
    # Feed it the TOA vector (size n) and Earth-to-SSB, Jupiter-to-SSB, etc.
    # (n,3) arrays. Set equatorial=True or False depending on the tempo2
    # coordinate frame, which matches the par-file coordinates.
    ct = 0

    # frame rotation (three angles, a rate, and an absolute offset)
    # use priors 1e-9, 5e-9, 5e-7, 1e-10, 1e-8, 5e-9, 1e-10
    # (based on systematic comparisons between ephemerides)
    earth = ss_framerotate(mjd, earth, 0.0, 0.0, 0.0, x[ct],
                           offset=None, equatorial=equatorial)
    ct += 1

    # jupiter
    earth = dmass(earth,jupiter,x[ct])
    ct += 1

    # saturn
    earth = dmass(earth,saturn,x[ct])
    ct += 1

    # uranus - uncertainty 3e-11, use twice that for prior (DE430-435 fit likes 6e-11)
    earth = dmass(earth,uranus,x[ct])
    ct += 1

    # neptune - uncertainty 8e-11, use twice that for prior (DE421-430 fit likes 6e-11 also)
    earth = dmass(earth,neptune,x[ct])
    ct += 1

    # Jupiter
    if incJuporb:
        if jup_orbmodel == 'angles':
            # rotate Jupiter (use 2e-8 prior for the three angles; no rate)
            earth = dorbit(mjd, earth, jupiter,
                           x[ct], x[ct+1], x[ct+2],
                           0.0, 0.0009547918983127075)
            ct += 3
        elif jup_orbmodel == 'orbelements':
            # perturb Jupiter's orbital elements with SVD partial design matrix
            jup_perturb_tmp = 0.0009547918983127075 * np.einsum('i,ijk->jk',
                                                                x[ct:ct+6],jup_orbelxyz)
            earth += np.array([np.interp(mjd, jup_mjd, jup_perturb_tmp[:,aa])
                               for aa in range(3)]).T
            ct += 6

    # Saturn
    if incSatorb:
        if sat_orbmodel == 'angles':
            # rotate Saturn (use 2e-8 prior for the three angles; no rate)
            earth = dorbit(mjd, earth, saturn,
                           x[ct], x[ct+1], x[ct+2],
                           0.0, 0.00028588567008942334)
            ct += 3
        if sat_orbmodel == 'orbelements':
            # perturb Saturn's orbital elements with SVD partial design matrix
            sat_perturb_tmp = 0.00028588567008942334 * np.einsum('i,ijk->jk',
                                                                x[ct:ct+6],sat_orbelxyz)
            earth += np.array([np.interp(mjd, sat_mjd, sat_perturb_tmp[:,aa])
                               for aa in range(3)]).T
            ct += 6


    return earth

# BayesEphem partials

In [ ]:
jup_mjd = np.load('./data/jupiter_orbitpartials/jupiter-orbel-mjd.npy')
jup_orbelxyz = np.load('./data/jupiter_orbitpartials/jupiter-orbel-xyz-svd.npy')

sat_mjd = np.load('./data/saturn_orbitpartials/saturn-orbel-mjd.npy')
sat_orbelxyz = np.load('./data/saturn_orbitpartials/saturn-orbel-xyz-svd.npy')

# Pulsar class

In [ ]:
class process_pulsar(object):
    
    def __init__(self, t2obj):
        
        self.T2psr = t2obj
        self.toas = np.double(self.T2psr.toas())
        self.res = np.double(self.T2psr.residuals())
        
        self.psrPos = self.T2psr.psrPos
        if 'ELONG' and 'ELAT' in self.T2psr.pars():
            # converting to equatorial
            print "--> Converting pulsar position time-series to equatorial"
            self.psrPos = utils.ecl2eq_vec(self.psrPos)

        # getting ephemeris properties
        self.ephemeris = self.T2psr.ephemeris
        if '436' in self.T2psr.ephemeris:
            self.ephemname = 'DE436'
        elif '435' in self.T2psr.ephemeris:
            self.ephemname = 'DE435'
        elif '430' in self.T2psr.ephemeris:
            self.ephemname = 'DE430'
        elif '421' in self.T2psr.ephemeris:
            self.ephemname = 'DE421'
        elif '418' in self.T2psr.ephemeris:
            self.ephemname = 'DE418'

        # populating roemer-delay dictionary
        self.roemer = OrderedDict()
        self.roemer[self.ephemname] = np.double(self.T2psr.roemer)

        # Planet position vectors will initially
        # be in coordinate system of .par file
        for ii in range(1,10):
            tag = 'DMASSPLANET'+str(ii)
            self.T2psr[tag].val = 0.0
        self.T2psr.formbats()
        self.planet_ssb = OrderedDict.fromkeys([self.ephemname])
        self.planet_ssb[self.ephemname] = np.zeros((len(self.T2psr.toas()),9,6))
        self.planet_ssb[self.ephemname][:,0,:] = self.T2psr.mercury_ssb
        self.planet_ssb[self.ephemname][:,1,:] = self.T2psr.venus_ssb
        self.planet_ssb[self.ephemname][:,2,:] = self.T2psr.earth_ssb
        self.planet_ssb[self.ephemname][:,3,:] = self.T2psr.mars_ssb
        self.planet_ssb[self.ephemname][:,4,:] = self.T2psr.jupiter_ssb
        self.planet_ssb[self.ephemname][:,5,:] = self.T2psr.saturn_ssb
        self.planet_ssb[self.ephemname][:,6,:] = self.T2psr.uranus_ssb
        self.planet_ssb[self.ephemname][:,7,:] = self.T2psr.neptune_ssb
        self.planet_ssb[self.ephemname][:,8,:] = self.T2psr.pluto_ssb

        if 'ELONG' and 'ELAT' in self.T2psr.pars():
            # Converting to equatorial if necessary
            print "--> Converting planet position time-series to equatorial"
            for ii in range(9):
                # position
                self.planet_ssb[self.ephemname][:,ii,:3] = \
                    utils.ecl2eq_vec(self.planet_ssb[self.ephemname][:,ii,:3])
                # velocity
                self.planet_ssb[self.ephemname][:,ii,3:] = \
                    utils.ecl2eq_vec(self.planet_ssb[self.ephemname][:,ii,3:])

        # get the sky position
        # check for B name
        if 'B' in self.T2psr.name:
            epoch = '1950'
        else:
            epoch = '2000'
        if 'RAJ' and 'DECJ' in self.T2psr.pars(which='set'):
            self.raj = np.double(self.T2psr['RAJ'].val)
            self.decj = np.double(self.T2psr['DECJ'].val)

            self.psr_locs = [self.raj, self.decj]

            eq = ephem.Equatorial(self.T2psr['RAJ'].val,
                                  self.T2psr['DECJ'].val)
            ec = ephem.Ecliptic(eq, epoch=epoch)
            self.elong = np.double(ec.lon)
            self.elat = np.double(ec.lat)

        elif 'ELONG' and 'ELAT' in self.T2psr.pars(which='set'):
            self.elong = np.double(self.T2psr['ELONG'].val)
            self.elat = np.double(self.T2psr['ELAT'].val)

            ec = ephem.Ecliptic(self.elong, self.elat)
            eq = ephem.Equatorial(ec, epoch=epoch)
            self.raj = np.double(eq.ra)
            self.decj = np.double(eq.dec)

            self.psr_locs = [self.raj, self.decj]

# Now calculate BayesEphem

In [ ]:
# Fill in these options

parfile = ...
timfile = ...
ephemeris = 'DE436'

In [ ]:
# Read in via libstempo
t2psr = T2.tempopulsar(parfile = parfile, timfile = timfile, 
                       maxobs=30000, ephem=ephemeris)

In [ ]:
psr = process_pulsar(t2psr)

In [ ]:
# first, construct the true geocenter to barycenter roemer
old_roemer = np.einsum('ij,ij->i', psr.planet_ssb[psr.ephemname][:,2,:3], psr.psrPos)

# now construct perturbation from physical model
tmp_earth = ssephem_physical_model(eph_physmodel_params, psr.toas,
                                   psr.planet_ssb[p.ephemname][:,2,:3], # earth
                                   psr.planet_ssb[p.ephemname][:,4,:3], # jupiter
                                   psr.planet_ssb[p.ephemname][:,5,:3], # saturn
                                   psr.planet_ssb[p.ephemname][:,6,:3], # uranus
                                   psr.planet_ssb[p.ephemname][:,7,:3], # neptune
                                   args.incJuporb, args.jup_orbmodel, jup_orbelxyz, jup_mjd,
                                   args.incSatorb, args.sat_orbmodel, sat_orbelxyz, sat_mjd,
                                   equatorial=True)
new_roemer = np.einsum('ij,ij->i', tmp_earth, psr.psrPos)

# subtract off old roemer, add in new one
#detres = psr.res
#detres -= old_roemer
#detres += new_roemer